# Evaluation


In [6]:
import pandas as pd

# Load logs
df = pd.read_csv("logs/trades.csv")

# Only completed trades (SELL)
exits = df[df["direction"] == "SELL"].copy()

# Basic stats
num_trades = len(exits)
total_pnl = exits["pnl"].sum()
win_trades = exits[exits["pnl"] > 0]
loss_trades = exits[exits["pnl"] < 0]

win_rate = len(win_trades) / num_trades * 100 if num_trades > 0 else 0

print("Number of completed trades:", num_trades)
print("Total PnL:", round(total_pnl, 2))
print("Win rate (%):", round(win_rate, 2))

# Optional: breakdown by asset
asset_summary = exits.groupby("asset")["pnl"].agg(
    trades="count",
    pnl="sum",
    avg_pnl="mean"
)

asset_summary


Number of completed trades: 24
Total PnL: 42.56
Win rate (%): 54.17


,trades,pnl,avg_pnl
asset,,,
BTC,11,15.357370,1.396125
ETH,13,27.199633,2.092279


# Short Analysis

### What appears to work 

-> The system successfully generates and executes trades based on a simple trend-following signal using EMA and momentum (RSI). 

-> Execution logic correctly transitions between flat and long positions, and exit rules (trend exit, take-profit, stop-loss) are respected. 

-> Logging captures sufficient detail (timestamps, prices, direction, position, exit reason) to evaluate performance and debug behavior. 

### What clearly does not 
-> The strategy shows sensitivity to short-term noise due to the short loop interval and loosened RSI threshold, leading to frequent trend exits and small losses.  
-> Profitability over the short test window is unstable and not statistically significant.  

### How I would iterate 
-> I would test longer EMA windows, add a volatility regime filter, and reduce trade frequency by requiring signal persistence across multiple intervals.  
-> On the execution side, I would explore limit-order placement and partial fills to reduce slippage.  
-> Finally, I would evaluate performance over longer periods and across additional assets.  